In [12]:
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

In [13]:
df = pd.read_excel(r'C:\Users\pawvaibh\Downloads\product classification.xlsx')
df1=df.copy()
df2=df1.copy()

In [14]:
df.drop_duplicates(inplace=True)
df1.drop_duplicates(inplace=True)

In [15]:
df.drop(columns=['ProductId', 'CustomerId', 'CustomerName', 'ProductName', 'SterilizationMethodName', 'MachineTypeName', 'ItemTypeName','SpecialtyName'], inplace=True)
df1.drop(columns=['ProductId', 'CustomerId', 'CustomerName', 'ProductName', 'SterilizationMethodName', 'MachineTypeName', 'ItemTypeName','SpecialtyName'], inplace=True)

In [11]:
# Use to_excel() method to save DataFrame to Excel
file_path = r'C:\Users\pawvaibh\Downloads\With_ProductId.xlsx'
df.to_excel(file_path, index=False)

In [16]:
df.head()

,SterilizationMethodId,MachineTypeId,ItemTypeId,SpecialtyId,CategoryName
0,26,2,4,31,Flexible Scope
13,0,2,4,31,Rigid Scope
18,0,2,4,31,Power System
27,26,2,4,31,Instrument
31,26,2,4,31,Rigid Scope


In [10]:
# Group by CategoryName and count occurrences
category_counts = df['CategoryName'].value_counts()
category_counts

CategoryName
Instrument        5501
Power System       943
Rigid Scope        845
Flexible Scope     274
Video              103
Ultrasound          74
Ophthalmic          24
Endocam             13
Name: count, dtype: int64

In [45]:
# Combine categories except "Instrument" into one category called "Other"
df.loc[~df['CategoryName'].isin(['Instrument']), 'CategoryName'] = 'Others'

In [46]:
# Calculate the total count of Instrument and Other categories
total_instrument_count = df[df['CategoryName'] == 'Instrument'].shape[0]
total_other_count = df[df['CategoryName'] == 'Others'].shape[0]

total_instrument_count, total_other_count

(5501, 2276)

In [47]:
# Calculate the target counts after downsampling
target_instrument_count = (total_other_count / 0.4) * 0.6
target_other_count = total_other_count

target_instrument_count, target_other_count

(3414.0, 2276)

In [48]:
# Downsample Instrument category
instrument_df = df[df['CategoryName'] == 'Instrument']
# removed_records_df = df[df['CategoryName'] != 'Instrument']  # Store removed records for validation
downsampled_instrument_df = resample(instrument_df, 
                                     replace=False,  # Without replacement
                                     n_samples=int(target_instrument_count),
                                     random_state=42)


In [49]:
# Combine downsampled Instrument category with Other category
final_df = pd.concat([downsampled_instrument_df, df[df['CategoryName'] == 'Others']])

In [50]:
final_df

,SterilizationMethodId,MachineTypeId,ItemTypeId,SpecialtyId,CategoryName
102072,4,2,5,32,Instrument
26622,10,2,4,57,Instrument
109974,14,2,4,33,Instrument
98249,41,2,4,22,Instrument
40035,53,2,4,32,Instrument
...,...,...,...,...,...
115735,14,1,4,61,Others
115776,53,1,4,32,Others
115784,2,1,1,27,Others
115817,14,1,4,32,Others


In [52]:
# Removed records (validation dataset)
removed_records_df = df[~df.index.isin(final_df.index)]
print("\nRemoved Records (Validation Dataset):")
print(removed_records_df)


Removed Records (Validation Dataset):
        SterilizationMethodId  MachineTypeId  ItemTypeId  SpecialtyId  \
175                         0              2           4           17   
212                        26              1           4           32   
232                        26              2           4           22   
328                        60              2           4           27   
369                        14              2           4           57   
...                       ...            ...         ...          ...   
115790                     14              2           4           32   
115878                     14              2           4           32   
115880                     14              2           4           27   
115891                     14              2           4           61   
115906                     14              2           4           61   

       CategoryName  
175      Instrument  
212      Instrument  
232      Instrumen

In [14]:
# Separate features and target variable
X = final_df.drop(columns=['CategoryName'])
y = final_df['CategoryName']

In [15]:
y

102072    Instrument
26622     Instrument
109974    Instrument
98249     Instrument
40035     Instrument
             ...    
115735        Others
115776        Others
115784        Others
115817        Others
115879        Others
Name: CategoryName, Length: 5690, dtype: object

In [16]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [17]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
# Train the first model
model_first = RandomForestClassifier(random_state=42)
model_first.fit(X, y)

# Predict using the first model
predictions_first = model_first.predict(X)

# Calculate accuracy
accuracy_first = accuracy_score(y, predictions_first)
print("Accuracy of first model:", accuracy_first)

Accuracy of first model: 0.8042179261862917


In [18]:
# Predict category for new input data
new_input = [[14, 2, 4, 32]]  # Example input data
predicted_category = model_first.predict(new_input)
print("Predicted Category:", predicted_category)

Predicted Category: ['Instrument']


C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [19]:
# Remove rows where CategoryName is 'Instrument'
df_sec = df1[df1['CategoryName'] != 'Instrument']

# Replace remaining CategoryNames with 'Others'
df_sec.loc[~df_sec['CategoryName'].isin(['Power System', 'Rigid Scope']), 'CategoryName'] = 'Others'

In [20]:
# Calculate the total count of Instrument and Other categories
total_ps_count = df_sec[df_sec['CategoryName'] == 'Power System'].shape[0]
total_rs_count = df_sec[df_sec['CategoryName'] == 'Rigid Scope'].shape[0]
total_other_count = df_sec[df_sec['CategoryName'] == 'Others'].shape[0]

total_ps_count, total_rs_count, total_other_count

(943, 845, 488)

In [21]:
# Separate features and target variable
X1 = df_sec.drop(columns=['CategoryName'])
y1 = df_sec['CategoryName']

In [22]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
# Train the sec model
model_sec = RandomForestClassifier(random_state=42)
model_sec.fit(X1, y1)

# Predict using the first model
predictions_first = model_sec.predict(X1)

# Calculate accuracy
accuracy_first = accuracy_score(y1, predictions_first)
print("Accuracy of first model:", accuracy_first)

Accuracy of first model: 0.7328646748681898


In [23]:
# Predict category for new input data
new_input = [[26, 1, 4, 57]]  # Example input data
predicted_category = model_sec.predict(new_input)
print("Predicted Category:", predicted_category)

Predicted Category: ['Rigid Scope']


C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [24]:
df3=df2.copy()

In [25]:
df2.drop_duplicates(inplace=True)
df2.drop(columns=['CustomerId','ProductId', 'CustomerName', 'ProductName', 'SterilizationMethodName', 'MachineTypeName', 'ItemTypeName','SpecialtyName'], inplace=True)

In [26]:
df_thi = df2[~df2['CategoryName'].isin(['Instrument', 'Rigid Scope', 'Power System'])]

# Replace remaining CategoryNames with 'Others' if not 'Flexible Scope' or 'Video'
df_thi.loc[~df_thi['CategoryName'].isin(['Flexible Scope', 'Video']), 'CategoryName'] = 'Others'

In [27]:
# Separate features and target variable
X2 = df_thi.drop(columns=['CategoryName'])
y2 = df_thi['CategoryName']

In [28]:
# Train the thi model
model_thi = RandomForestClassifier(random_state=42)
model_thi.fit(X2, y2)

# Predict using the first model
predictions_first = model_thi.predict(X2)

# Calculate accuracy
accuracy_first = accuracy_score(y2, predictions_first)
print("Accuracy of first model:", accuracy_first)

Accuracy of first model: 0.8688524590163934


In [29]:
# Predict category for new input data
new_input = [[41, 2, 4, 22]]  # Example input data
predicted_category = model_thi.predict(new_input)
print("Predicted Category:", predicted_category)

Predicted Category: ['Others']


C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [30]:
df4=df3.copy()
df3.drop_duplicates(inplace=True)
df3.drop(columns=['CustomerId','ProductId', 'CustomerName', 'ProductName', 'SterilizationMethodName', 'MachineTypeName', 'ItemTypeName','SpecialtyName'], inplace=True)

In [31]:
df_last = df3[df3['CategoryName'].isin(['Ultrasound', 'Opthamlmic', 'Endocam'])]

In [32]:
# Separate features and target variable
X3 = df_last.drop(columns=['CategoryName'])
y3 = df_last['CategoryName']

In [33]:
# Train the thi model
model_last = RandomForestClassifier(random_state=42)
model_last.fit(X3, y3)

# Predict using the first model
predictions_first = model_last.predict(X3)

# Calculate accuracy
accuracy_first = accuracy_score(y3, predictions_first)
print("Accuracy of first model:", accuracy_first)

Accuracy of first model: 0.9770114942528736


In [34]:
# Predict category for new input data
new_input = [[14, 2, 4, 61]]  # Example input data
predicted_category = model_last.predict(new_input)
print("Predicted Category:", predicted_category)

Predicted Category: ['Endocam']


C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [35]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.metrics import accuracy_score

# Read the data
df = pd.read_excel(r'C:\Users\pawvaibh\Downloads\product classification.xlsx')

# Function to preprocess data
def preprocess_data(df):
    df.drop_duplicates(inplace=True)
    df.drop(columns=['CustomerId','ProductId', 'CustomerName', 'ProductName', 'SterilizationMethodName', 'MachineTypeName', 'ItemTypeName','SpecialtyName'], inplace=True)
    return df

# Combine categories except "Instrument" into one category called "Others"
def combine_categories(df):
    df.loc[~df['CategoryName'].isin(['Instrument']), 'CategoryName'] = 'Others'
    return df

# Downsample Instrument category
def downsample_instrument(df, target_instrument_count):
    instrument_df = df[df['CategoryName'] == 'Instrument']
    downsampled_instrument_df = resample(instrument_df, replace=False, n_samples=int(target_instrument_count), random_state=42)
    final_df = pd.concat([downsampled_instrument_df, df[df['CategoryName'] == 'Others']])
    return final_df

#Train and test split
def train_and_test(df):
    train_df, test_df = train_test_split(df, test_size=0.4, random_state=42)
    return train_df, test_df

def segregate(train_df, test_df):
    X_train = train_df.drop(columns=['CategoryName'])
    y_train = train_df['CategoryName']
    X_test = test_df.drop(columns=['CategoryName'])
    y_test = test_df['CategoryName']
    return X_train, y_train, X_test, y_test
    
# Train and evaluate RandomForestClassifier
def train_and_evaluate_model(X_train, y_train, X_test, y_test):
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    
    # Predict on the training and testing data
    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)
    
    # Calculate accuracy for training and testing data
    train_accuracy = accuracy_score(y_train, train_predictions)
    test_accuracy = accuracy_score(y_test, test_predictions)

    precision = precision_score(y_test, test_predictions, average='weighted')
    recall = recall_score(y_test, test_predictions, average='weighted')
    f1 = f1_score(y_test, test_predictions, average='weighted')
    
    print("Training Accuracy:", train_accuracy)
    print("Testing Accuracy:", test_accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    
    return model, test_predictions

# Preprocess the data
df1 = preprocess_data(df.copy())
df2 = preprocess_data(df.copy())
df3 = preprocess_data(df.copy())
df4 = preprocess_data(df.copy())

# Model 1: Classify Instrument vs. Others
df1 = combine_categories(df1)

# Calculate target instrument count
total_instrument_count = df1[df1['CategoryName'] == 'Instrument'].shape[0]
total_other_count = df1[df1['CategoryName'] == 'Others'].shape[0]
target_instrument_count = (total_other_count / 0.4) * 0.6

df1 = downsample_instrument(df1, target_instrument_count)

# Split data into train and test sets
train_df1, test_df1 = train_and_test(df1)

X1_train, y1_train, X1_test, y1_test = segregate(train_df1, test_df1)

print("Model 1:")

model_first, test_predictions_1 = train_and_evaluate_model(X1_train, y1_train, X1_test, y1_test)

# Predict category for new input data
new_input = [[53, 1, 4, 40]]  # Example input data #53,1,4,40
predicted_category = model_first.predict(new_input)
print("Predicted Category from Model 1:", predicted_category)

if predicted_category == 'Others':
    # Model 2: Classify Power System, Rigid Scope vs. Others
    df2 = df2[~df2['CategoryName'].isin(['Instrument'])]
    df2.loc[~df2['CategoryName'].isin(['Power System', 'Rigid Scope']), 'CategoryName'] = 'Others'

    train_df2, test_df2 = train_and_test(df2)

    X2_train, y2_train, X2_test, y2_test = segregate(train_df2, test_df2)
 
    print("\nModel 2:")
    model_sec, test_predictions_2 = train_and_evaluate_model(X2_train, y2_train, X2_test, y2_test)
    
    predicted_category = model_sec.predict(new_input)
    print("Predicted Category from Model 2:", predicted_category)
    
    if predicted_category == 'Others':
        # Model 3: Classify Flexible Scope, Video Vs. Others
        df3 = df3[~df3['CategoryName'].isin(['Instrument', 'Rigid Scope', 'Power System'])]
        df3.loc[~df3['CategoryName'].isin(['Flexible Scope', 'Video']), 'CategoryName'] = 'Others'

        train_df3, test_df3 = train_and_test(df3)

        X3_train, y3_train, X3_test, y3_test = segregate(train_df3, test_df3)
     
        print("\nModel 3:")
        model_third, test_predictions_3 = train_and_evaluate_model(X3_train, y3_train, X3_test, y3_test)
        
        predicted_category = model_third.predict(new_input)
        print("Final Predicted Category from Model 3:", predicted_category)

        if predicted_category == 'Others':
            # Model 4: Classify 'Ultrasound', 'Opthamlmic', 'Endocam'
            df4 = df4[df4['CategoryName'].isin(['Ultrasound', 'Opthamlmic', 'Endocam'])]

            train_df4, test_df4 = train_and_test(df4)

            X4_train, y4_train, X4_test, y4_test = segregate(train_df4, test_df4)
  
            print("\nModel 4:")
            model_last, test_predictions_4 = train_and_evaluate_model(X4_train, y4_train, X4_test, y4_test)
        
            predicted_category = model_last.predict(new_input)
            print("Final Predicted Category from Model 4:", predicted_category)

        else:
            print("Final Predicted Category from Model 3:", predicted_category)
    else:
        print("Final Predicted Category from Model 2:", predicted_category)
else:
    print("Final Predicted Category from Model 1:", predicted_category)
    

Model 1:
Training Accuracy: 0.8075571177504394
Testing Accuracy: 0.7671353251318102
Precision: 0.7676402508465207
Recall: 0.7671353251318102
F1 Score: 0.7589518522880316
Predicted Category from Model 1: ['Others']

Model 2:


C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Training Accuracy: 0.7611721611721611
Testing Accuracy: 0.5828759604829857
Precision: 0.5720387461916867
Recall: 0.5828759604829857
F1 Score: 0.5729608781999274
Predicted Category from Model 2: ['Others']

Model 3:
Training Accuracy: 0.8767123287671232
Testing Accuracy: 0.6530612244897959
Precision: 0.6392447088935735
Recall: 0.6530612244897959
F1 Score: 0.641194659237876
Final Predicted Category from Model 3: ['Others']

Model 4:


C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Training Accuracy: 0.9807692307692307
Testing Accuracy: 0.8
Precision: 0.813265306122449
Recall: 0.8
F1 Score: 0.8059379217273954
Final Predicted Category from Model 4: ['Endocam']


C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [36]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.metrics import accuracy_score

# Read the data
df = pd.read_excel(r'C:\Users\pawvaibh\Downloads\product classification.xlsx')

# Function to preprocess data
def preprocess_data(df):
    df.drop_duplicates(inplace=True)
    df.drop(columns=['CustomerId','ProductId', 'CustomerName', 'ProductName', 'SterilizationMethodName', 'MachineTypeName', 'ItemTypeName','SpecialtyName'], inplace=True)
    return df

# Combine categories except "Instrument" into one category called "Others"
def combine_categories(df):
    df.loc[~df['CategoryName'].isin(['Instrument']), 'CategoryName'] = 'Others'
    return df

# Downsample Instrument category
def downsample_instrument(df, target_instrument_count):
    instrument_df = df[df['CategoryName'] == 'Instrument']
    downsampled_instrument_df = resample(instrument_df, replace=False, n_samples=int(target_instrument_count), random_state=42)
    final_df = pd.concat([downsampled_instrument_df, df[df['CategoryName'] == 'Others']])
    return final_df

# Train and evaluate RandomForestClassifier
def train_and_evaluate_model(X, y):
    model = RandomForestClassifier(random_state=42)
    model.fit(X, y)
    predictions = model.predict(X)
    accuracy = accuracy_score(y, predictions)
    print("Accuracy:", accuracy)
    return model

# Preprocess the data
df1 = preprocess_data(df.copy())
df2 = preprocess_data(df.copy())
df3 = preprocess_data(df.copy())
df4 = preprocess_data(df.copy())

# Model 1: Classify Instrument vs. Others
df1 = combine_categories(df1)

# Calculate target instrument count
total_instrument_count = df1[df1['CategoryName'] == 'Instrument'].shape[0]
total_other_count = df1[df1['CategoryName'] == 'Others'].shape[0]
target_instrument_count = (total_other_count / 0.4) * 0.6

df1 = downsample_instrument(df1, target_instrument_count)
X1 = df1.drop(columns=['CategoryName'])
y1 = df1['CategoryName']
print("Model 1:")
model_first = train_and_evaluate_model(X1, y1)

# Model 2: Classify Power System, Rigid Scope vs. Others
df2 = df2[~df2['CategoryName'].isin(['Instrument'])]
df2.loc[~df2['CategoryName'].isin(['Power System', 'Rigid Scope']), 'CategoryName'] = 'Others'
X2 = df2.drop(columns=['CategoryName'])
y2 = df2['CategoryName']
print("\nModel 2:")
model_sec = train_and_evaluate_model(X2, y2)

# Model 3: Classify Flexible Scope, Video Vs. Others
df3 = df3[~df3['CategoryName'].isin(['Instrument', 'Rigid Scope', 'Power System'])]
df3.loc[~df3['CategoryName'].isin(['Flexible Scope', 'Video']), 'CategoryName'] = 'Others'
X3 = df3.drop(columns=['CategoryName'])
y3 = df3['CategoryName']
print("\nModel 3:")
model_third = train_and_evaluate_model(X3, y3)

# Model 4: Classify 'Ultrasound', 'Opthamlmic', 'Endocam'
df4 = df4[df4['CategoryName'].isin(['Ultrasound', 'Opthamlmic', 'Endocam'])]
X4 = df4.drop(columns=['CategoryName'])
y4 = df4['CategoryName']
print("\nModel 4:")
model_last = train_and_evaluate_model(X4, y4)

# Function to predict category
def predict_category(input_data):
    # Model 1
    predicted_category = model_first.predict(input_data)
    print("Predicted Category from Model 1:", predicted_category)
    if 'Others' in predicted_category:
        # Model 2
        predicted_category = model_sec.predict(input_data)
        print("Predicted Category from Model 2:", predicted_category)
        if 'Others' in predicted_category:
            # Model 3
            predicted_category = model_third.predict(input_data)
            print("Predicted Category from Model 3:", predicted_category)
            if 'Others' in predicted_category:
                # Model 4
                predicted_category = model_last.predict(input_data)
                print("Final Predicted Category from Model 4:", predicted_category)
            else:
                print("Final Predicted Category from Model 3:", predicted_category)
        else:
            print("Final Predicted Category from Model 2:", predicted_category)
    else:
        print("Final Predicted Category from Model 1:", predicted_category)

# Predict category for new input data
new_input = [[14, 2, 4, 61]]  # Example input data
predict_category(new_input)


Model 1:
Accuracy: 0.8042179261862917

Model 2:
Accuracy: 0.7328646748681898

Model 3:
Accuracy: 0.8688524590163934

Model 4:
Accuracy: 0.9770114942528736
Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']


C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [37]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.metrics import accuracy_score

# Read the data
df = pd.read_excel(r'C:\Users\pawvaibh\Downloads\product classification.xlsx')

# Function to preprocess data
def preprocess_data(df):
    df.drop_duplicates(inplace=True)
    df.drop(columns=['CustomerId','ProductId', 'CustomerName', 'ProductName', 'SterilizationMethodName', 'MachineTypeName', 'ItemTypeName','SpecialtyName'], inplace=True)
    return df

# Combine categories except "Instrument" into one category called "Others"
def combine_categories(df):
    df1 = df.loc[~df['CategoryName'].isin(['Instrument']), 'CategoryName'] = 'Others'
    return df1

# Downsample Instrument category
def downsample_instrument(df, target_instrument_count):
    instrument_df = df[df['CategoryName'] == 'Instrument']
    downsampled_instrument_df = resample(instrument_df, replace=False, n_samples=int(target_instrument_count), random_state=42)
    final_df = pd.concat([downsampled_instrument_df, df[df['CategoryName'] == 'Others']])
    return final_df

# Train and evaluate RandomForestClassifier
def train_and_evaluate_model(X, y):
    model = RandomForestClassifier(random_state=42)
    model.fit(X, y)
    predictions = model.predict(X)
    accuracy = accuracy_score(y, predictions)
    print("Accuracy:", accuracy)
    return model, predictions

# Preprocess the data
df = preprocess_data(df.copy())
# df2 = preprocess_data(df.copy())
# df3 = preprocess_data(df.copy())
# df4 = preprocess_data(df.copy())

# Model 1: Classify Instrument vs. Others
df1 = combine_categories(df)

# Calculate target instrument count
total_instrument_count = df1[df1['CategoryName'] == 'Instrument'].shape[0]
total_other_count = df1[df1['CategoryName'] == 'Others'].shape[0]
target_instrument_count = (total_other_count / 0.4) * 0.6

df1 = downsample_instrument(df1, target_instrument_count)
X1 = df1.drop(columns=['CategoryName'])
y1 = df1['CategoryName']
print("Model 1:")
model_first, _ = train_and_evaluate_model(X1, y1)

# # Predict category for new input data
# new_input = [[53, 1, 4, 40]]  # Example input data
# predicted_category = model_first.predict(new_input)
# print("Predicted Category from Model 1:", predicted_category)
print(df)
if predicted_category == 'Others':
    # Model 2: Classify Power System, Rigid Scope vs. Others
    df2 = df[~df['CategoryName'].isin(['Instrument'])]
    df2.loc[~df2['CategoryName'].isin(['Power System', 'Rigid Scope']), 'CategoryName'] = 'Others'
    X2 = df2.drop(columns=['CategoryName'])
    y2 = df2['CategoryName']
    print("\nModel 2:")
    model_sec, _ = train_and_evaluate_model(X2, y2)
    
    predicted_category = model_sec.predict(new_input)
    print("Predicted Category from Model 2:", predicted_category)
    
    if predicted_category == 'Others':
        # Model 3: Classify Flexible Scope, Video Vs. Others
        df3 = df[~df['CategoryName'].isin(['Instrument', 'Rigid Scope', 'Power System'])]
        df3.loc[~df3['CategoryName'].isin(['Flexible Scope', 'Video']), 'CategoryName'] = 'Others'
        X3 = df3.drop(columns=['CategoryName'])
        y3 = df3['CategoryName']
        print("\nModel 3:")
        model_third, _ = train_and_evaluate_model(X3, y3)
        
        predicted_category = model_third.predict(new_input)
        print("Final Predicted Category from Model 3:", predicted_category)

        if predicted_category == 'Others':
            # Model 4: Classify 'Ultrasound', 'Opthamlmic', 'Endocam'
            df4 = df[df['CategoryName'].isin(['Ultrasound', 'Opthamlmic', 'Endocam'])]
            X4 = df4.drop(columns=['CategoryName'])
            y4 = df4['CategoryName']
            print("\nModel 4:")
            model_last, _ = train_and_evaluate_model(X4, y4)
        
            predicted_category = model_last.predict(new_input)
            print("Final Predicted Category from Model 4:", predicted_category)

        else:
            print("Final Predicted Category from Model 3:", predicted_category)
    else:
        print("Final Predicted Category from Model 2:", predicted_category)
else:
    print("Final Predicted Category from Model 1:", predicted_category)

TypeError: string indices must be integers, not 'str'